In [1]:
import os
import json
import time
import requests
import pandas as pd
from pathlib import Path
from tqdm import tqdm

# === Claude API setup ===
api_key = "api-key"
api_url = "https://api.anthropic.com/v1/messages"
model_name = "claude-3-haiku-20240307"

headers = {
    "x-api-key": api_key,
    "anthropic-version": "2023-06-01",
    "content-type": "application/json",
}

# === Paths ===
base = Path.home() / "Desktop" / "Benchmark"
subs_dir = base / "json_benchmark"
summaries_dir = base / "summaries"
output_dir = base / "scored_progressive_women"
prompts_dir = base / "prompts_progressive_women"

output_dir.mkdir(parents=True, exist_ok=True)
prompts_dir.mkdir(parents=True, exist_ok=True)

matches_df = pd.read_csv(base / "matches_benchmark.csv")

# === Prompt template ===
def create_progressive_women_prompt(summary, chunk):
    return f"""
You are evaluating how progressively women are portrayed in a film based on its plot summary and dialogue.

Your task is to classify the film into one of the following categories based on the roles, agency, and representation of women:

- **Yes**: The film offers a clearly progressive portrayal of women. Female characters are active agents, hold central narrative roles, and are treated with depth and respect.
- **Mixed**: The portrayal includes both progressive and stereotypical elements. Some female characters may show agency, while others may be sidelined or underdeveloped.
- **No**: The film does not offer a progressive portrayal. Female characters are absent, objectified, stereotyped, or lack meaningful agency.

Please:
1. Choose the best label: "yes", "mixed", or "no"
2. Provide a short justification (1–2 sentences)
3. Include a confidence score between 0.0 and 1.0

Return your answer in this exact JSON format:

{{
  "progressive_women": "...",
  "confidence": ...,
  "explanation": "..."
}}

Film Summary:
\"\"\"{summary}\"\"\"

Dialogue:
\"\"\"{chunk}\"\"\"
""".strip()

# === Claude API call ===
def call_claude(prompt):
    payload = {
        "model": model_name,
        "max_tokens": 500,
        "temperature": 0,
        "system": "You are a careful film analyst. Always return exactly and only the required JSON.",
        "messages": [{"role": "user", "content": prompt}]
    }

    response = requests.post(api_url, headers=headers, json=payload)

    if response.status_code == 429:
        print("⚠️ Rate limit hit. Waiting...")
        time.sleep(60)
        return call_claude(prompt)
    elif response.status_code != 200:
        print(f"❌ API Error: {response.status_code}")
        return None

    try:
        content = response.json()["content"][0]["text"]
        return json.loads(content)
    except Exception:
        print(f"⚠️ JSON parsing error:\n{response.text}")
        return None

# === Chunk subtitles ===
def chunk_dialogue(subs, chunk_size=5000, overlap=200):
    text_blocks = [line.get("text", "").strip() for line in subs if line.get("text")]
    full_text = " ".join(text_blocks)

    chunks = []
    start = 0
    while start < len(full_text):
        end = start + chunk_size
        chunks.append(full_text[start:end])
        start = end - overlap
    return chunks

# === Scoring loop ===
for _, row in tqdm(matches_df.iterrows(), total=len(matches_df), desc="Scoring progressive_women"):
    filename = row["subtitle_filename"]
    json_path = subs_dir / f"{filename}.json"
    summary_path = summaries_dir / f"{filename}.srt_summary.txt"

    if not json_path.exists() or not summary_path.exists():
        print(f"⚠️ Missing files for: {filename}")
        continue

    with open(json_path, "r", encoding="utf-8") as f:
        subs = json.load(f)

    with open(summary_path, "r", encoding="utf-8") as f:
        summary = f.read().strip()

    chunks = chunk_dialogue(subs)

    for i, chunk in enumerate(chunks):
        output_path = output_dir / f"{filename}_chunk{i+1}_progressive_women.json"
        prompt_path = prompts_dir / f"{filename}_chunk{i+1}_progressive_women_prompt.txt"

        if output_path.exists():
            print(f"🟡 Already scored: {filename} chunk {i+1}")
            continue

        prompt = create_progressive_women_prompt(summary, chunk)

        with open(prompt_path, "w", encoding="utf-8") as f:
            f.write(prompt)

        result = call_claude(prompt)
        if result:
            with open(output_path, "w", encoding="utf-8") as f:
                json.dump(result, f, indent=2)
            print(f"✅ Scored: {filename} chunk {i+1}")
        else:
            print(f"❌ Failed: {filename} chunk {i+1}")

        time.sleep(1.5)


Scoring progressive_women:   0%|                         | 0/20 [00:00<?, ?it/s]

✅ Scored: 2006Blood.Diamond chunk 1
✅ Scored: 2006Blood.Diamond chunk 2
✅ Scored: 2006Blood.Diamond chunk 3
✅ Scored: 2006Blood.Diamond chunk 4
✅ Scored: 2006Blood.Diamond chunk 5
✅ Scored: 2006Blood.Diamond chunk 6
✅ Scored: 2006Blood.Diamond chunk 7
✅ Scored: 2006Blood.Diamond chunk 8
✅ Scored: 2006Blood.Diamond chunk 9
✅ Scored: 2006Blood.Diamond chunk 10
✅ Scored: 2006Blood.Diamond chunk 11


Scoring progressive_women:   5%|▊                | 1/20 [00:47<14:55, 47.15s/it]

✅ Scored: 2005The.Constant.Gardener chunk 1
✅ Scored: 2005The.Constant.Gardener chunk 2
✅ Scored: 2005The.Constant.Gardener chunk 3
✅ Scored: 2005The.Constant.Gardener chunk 4
✅ Scored: 2005The.Constant.Gardener chunk 5
✅ Scored: 2005The.Constant.Gardener chunk 6
✅ Scored: 2005The.Constant.Gardener chunk 7
✅ Scored: 2005The.Constant.Gardener chunk 8
✅ Scored: 2005The.Constant.Gardener chunk 9
✅ Scored: 2005The.Constant.Gardener chunk 10
✅ Scored: 2005The.Constant.Gardener chunk 11
✅ Scored: 2005The.Constant.Gardener chunk 12
✅ Scored: 2005The.Constant.Gardener chunk 13


Scoring progressive_women:  10%|█▋               | 2/20 [01:26<12:51, 42.85s/it]

...

Scoring progressive_women:  90%|██████████████▍ | 18/20 [10:42<01:28, 44.11s/it]

✅ Scored: 1985Back.To.The.Future chunk 1
✅ Scored: 1985Back.To.The.Future chunk 2
✅ Scored: 1985Back.To.The.Future chunk 3
✅ Scored: 1985Back.To.The.Future chunk 4
✅ Scored: 1985Back.To.The.Future chunk 5
✅ Scored: 1985Back.To.The.Future chunk 6
✅ Scored: 1985Back.To.The.Future chunk 7
✅ Scored: 1985Back.To.The.Future chunk 8
✅ Scored: 1985Back.To.The.Future chunk 9
✅ Scored: 1985Back.To.The.Future chunk 10
✅ Scored: 1985Back.To.The.Future chunk 11


Scoring progressive_women:  95%|███████████████▏| 19/20 [11:15<00:40, 40.94s/it]

✅ Scored: 2013The.Purge chunk 1
✅ Scored: 2013The.Purge chunk 2
✅ Scored: 2013The.Purge chunk 3
✅ Scored: 2013The.Purge chunk 4
✅ Scored: 2013The.Purge chunk 5
✅ Scored: 2013The.Purge chunk 6
✅ Scored: 2013The.Purge chunk 7


Scoring progressive_women: 100%|████████████████| 20/20 [13:35<00:00, 40.77s/it]


In [3]:
import os
import json
import pandas as pd
from pathlib import Path
from collections import defaultdict, Counter

# === Paths ===
base_path = Path.home() / "Desktop" / "Benchmark"
scored_dir = base_path / "scored_progressive_women"
output_csv_path = base_path / "model_progressive_women_output.csv"

# === Aggregate Results ===
aggregated_results = []

# Group chunk files by movie
movie_files = defaultdict(list)
for file in scored_dir.glob("*_chunk*_progressive_women.json"):
    movie_id = file.name.split("_chunk")[0]
    movie_files[movie_id].append(file)

# Process each movie's chunks
for movie_id, files in movie_files.items():
    label_counts = Counter()
    confidence_sums = defaultdict(float)
    confidence_counts = defaultdict(int)
    explanations = []

    for file in files:
        with open(file, "r", encoding="utf-8") as f:
            try:
                data = json.load(f)
                label = data.get("progressive_women", "").strip().lower()
                confidence = float(data.get("confidence", 0.0))
                explanation = data.get("explanation", "")

                label_counts[label] += 1
                confidence_sums[label] += confidence
                confidence_counts[label] += 1
                explanations.append(f"{label} ({confidence:.2f}): {explanation}")

            except Exception as e:
                print(f"⚠️ Error in file {file.name}: {e}")

    # Compute average confidence per label
    avg_confidences = {
        label: round(confidence_sums[label] / confidence_counts[label], 3)
        for label in label_counts
    }

    # Weighted vote = label with highest total confidence
    weighted_vote = max(confidence_sums.items(), key=lambda x: x[1])[0] if confidence_sums else None

    aggregated_results.append({
        "subtitle_filename": movie_id,
        "label_counts": dict(label_counts),
        "avg_confidences": avg_confidences,
        "weighted_vote": weighted_vote,
        "explanations": explanations
    })

# === Save to CSV ===
df = pd.DataFrame(aggregated_results)
df.to_csv(output_csv_path, index=False)
print(f"✅ Saved aggregated results to: {output_csv_path}")


✅ Saved aggregated results to: /Users/cedricroetheli/Desktop/Benchmark/model_progressive_women_output.csv


In [5]:
import pandas as pd
from pathlib import Path

# === Paths ===
base = Path.home() / "Desktop" / "Benchmark"
truth_path = base / "benchmark_final.csv"
model_path = base / "model_progressive_women_output.csv"
output_path = base / "progressive_women_evaluation.csv"

# === Load Data ===
truth_df = pd.read_csv(truth_path)
model_df = pd.read_csv(model_path)

# === Normalize filenames ===
truth_df["subtitle_filename"] = truth_df["subtitle_filename"].str.strip()
model_df["subtitle_filename"] = model_df["subtitle_filename"].str.strip()

# === Label Mapping for Consistency ===
label_map = {
    "yes": "yes",
    "no": "no",
    "mixed": "mixed",
    "partially": "mixed",
    "partly": "mixed"
}

def normalize_label(label):
    if pd.isna(label):
        return ""
    label = label.lower().strip()
    return label_map.get(label, label)

def normalize_set(label_str):
    if pd.isna(label_str):
        return set()
    parts = [normalize_label(part) for part in label_str.split("|")]
    return set(parts)

truth_df["progressive_women_set"] = truth_df["progressive_women"].apply(normalize_set)
model_df["normalized_vote"] = model_df["weighted_vote"].apply(normalize_label)

# === Merge and Evaluate ===
merged_df = pd.merge(model_df, truth_df, on="subtitle_filename", how="inner")
merged_df["is_correct"] = merged_df.apply(
    lambda row: row["normalized_vote"] in row["progressive_women_set"], axis=1
)

# === Output CSV ===
evaluation_df = merged_df[[
    "subtitle_filename", "progressive_women", "normalized_vote", "is_correct"
]].copy()
evaluation_df.columns = ["movie", "benchmark_progressive_women", "model_progressive_women", "is_correct"]
evaluation_df.to_csv(output_path, index=False)

# === Summary ===
total = len(evaluation_df)
correct = evaluation_df["is_correct"].sum()
accuracy = correct / total if total else 0

print(f"🎯 Evaluation complete:")
print(f"✅ Correct: {correct}/{total}")
print(f"📊 Accuracy: {accuracy:.2%}")
print(f"📁 Saved to: {output_path}")


🎯 Evaluation complete:
✅ Correct: 16/20
📊 Accuracy: 80.00%
📁 Saved to: /Users/cedricroetheli/Desktop/Benchmark/progressive_women_evaluation.csv
